# Solar Panel Dataset
## Company: EasyGreen
### Explanation of Variables in Dataset

totalUsePower: Total kWh (amount of electricity) used today.


totalProductPower: Total kWh (amount of electricity) produced by the solar panels today.


totalSelfUsePower: Total kWh (amount of electricity) of today's production used for home electricity consumption.


totalBuyPower: Total kWh (amount of electricity) bought from the grid today.

In [5]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pyairtable import Api
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Connection details
username = 'introToML'
password = 'pddbb1c530308955bbd2c5aa59a647e30074fc7b7ebde93f626c481f4de93f7de'
host = 'ec2-63-32-137-56.eu-west-1.compute.amazonaws.com'
port = '5432'
dbname = 'dcsdgo449thll5'

# Connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'

# Create the engine
engine = create_engine(connection_string)

# Specify your table name
table_name = 'power_usage'

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, engine, schema='public')

In [6]:
addressData = pd.read_csv('addressData.csv')
# rename Kunde ID to user_id, Kundens navn to name, CPR adresse to address
addressData.rename(columns={'Kunde ID':'user_id', 'Kundens navn':'name', 'CPR adresse':'address'}, inplace=True)
addressData['user_id'] = addressData['user_id'].fillna(0).astype(int)

In [7]:
def clean_addresses(address):
    # Replace unwanted characters with a space or remove them
    address = address.str.replace(r'[@#*]', '', regex=True)  # Remove @, #, *
    address = address.str.replace(r'[\n\r]', ' ', regex=True)  # Replace newlines with space
    address = address.str.strip()  # Strip leading/trailing whitespace
    return address

# Apply cleaning function
addressData['address'] = clean_addresses(addressData['address'])

In [19]:
import requests
import time 

def geocode_address(api_key, address):
    base_url = "https://us1.locationiq.com/v1/search.php"
    params = {
        "key": api_key,
        "q": address,
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 0:
            lat = data[0]['lat']
            lon = data[0]['lon']
            print(f"Address: {address} -> Latitude: {lat}, Longitude: {lon}")
            time.sleep(1)
            return lat, lon
        else:
            print(f"Address: {address} -> No results found.")
            return None, None
        
    else:
        print(f"Address: {address} -> Failed to geocode with status code {response.status_code}.")
        time.sleep(1)
        return None, None    
        
def apply_geocoding(row):
    lat, lon = geocode_address("pk.fdf56d0c2c0bbfc3b4054a3a45c0bd72", row['address'])
    return pd.Series([lat, lon])

In [ ]:
# Applying geocoding
addressData[['latitude', 'longitude']] = addressData.apply(apply_geocoding, axis=1)

addressData.head()

In [22]:
addressData.to_csv('addressData.csv', index=False)

In [23]:
# merge addressData and df on user_id

dfMerged = df.merge(addressData, on='user_id', how='left')

In [26]:
dfMerged.to_csv('dfMerged.csv', index=False)